# $\color{#dda}{\text{FIC: FLOPs Information Criterion}}$ 
## $\color{#dda}{\text{(Remi's Information Criterion - RIC)}}$

**Autor:** Remi Heredia  
**Fecha:** Octubre 2025  
**Versión:** 2.0 (Fórmula Paramétrica Optimizada)

## $\color{#dda}{\text{1. Motivación}}$

Los criterios de información tradicionales (AIC, BIC, MDL) penalizan la complejidad del modelo mediante el **número de parámetros**. Sin embargo, en deep learning moderno:

- Dos modelos con igual número de parámetros pueden tener **complejidad computacional muy diferente**
- Un modelo con más parámetros puede ser más eficiente (ej: MobileNet vs ResNet)
- La arquitectura (depth, skip connections, etc.) no se refleja en el conteo de parámetros

Además, con el avance tecnológico y el uso de modelos tan pesados para la realización de tareas relativamente simples, considero que se debe contemplar el consumo energético. Espero que mi enfoque pueda ser de utilidad para ayudar a mitigar al menos un poco el impacto ambiental que tienen (sobre todo) los modelos más pesados.

La optimización de modelos operacionalmente podría ayudar a desarrollar herramientas más compactas y, al menos en materia de NLPs, más del estilo "sistema experto" en vez de "todólogos" (que es lo que está de moda). La problemática por la que se me ocurrió crear esto fue esencialmente porque quiero hostear mis propios GPTs en monoplacas; Quiero entrenar y testear mis modelos de lenguaje natural sin demasiado poder computacional ni una cantidad ingente de memoria.

**Ejemplo ilustrativo:**

| Modelo | Parámetros | FLOPs | AIC/BIC | FIC |
|--------|-----------|-------|---------|-----|
| Modelo A | 10M | 1 GFLOP | Penaliza 10M | Penaliza 10M + 1G |
| Modelo B | 10M | 5 GFLOPs | Penaliza 10M (igual) | Penaliza 10M + 5G |

**Problema:** AIC y BIC asignan la misma penalización a ambos modelos, ignorando que el Modelo B requiere 5× más cómputo.

**Propuesta:** Penalizar la complejidad mediante **FLOPs** además de parámetros, capturando así el costo computacional real de ejecutar el modelo.

## $\color{#dda}{\text{2. Definición General del FIC}}$


El **FLOPs Information Criterion (FIC)**, también conocido como **RIC (Remi's Information Criterion porque así me llamo)**, es un criterio de selección de modelos que penaliza tanto el desajuste a los datos como la complejidad computacional medida en FLOPs.

**Principio fundamental:**
> En producción, los modelos se ejecutan millones de veces. El costo computacional importa tanto como el ajuste estadístico.

## $\color{#dda}{\text{3. Relación con Big O y Complejidad Algorítmica}}$


Para evitar preguntas de complejidad computacional, espacio en memoria y criterios como "yo elijo modelos en base al tiempo que tardan en correr", hice una pequeña explicación de por qué la notación *Big O* **NO** se puede aplicar a la clase de modelos con los que se trabaja en la ciencia de datos y el aprendizaje máquina.

Explico todo con bolitas y palitos porque no sé qué tan multidisciplinario pueda ser considerado este proyecto.

### $\color{#dda}{\text{3.1 Big O vs FLOPs: Diferencias Fundamentales}}$

| Aspecto | Big O | FLOPs (FIC) |
|---------|-------|-------------|
| **Naturaleza** | Teórica/Asintótica | Práctica/Exacta |
| **Pregunta** | "¿Cómo escala con $n$?" | "¿Cuántas operaciones?" |
| **Output** | $O(n^2)$, $O(n \log n)$ | 2,000,000 FLOPs |
| **Constantes** | Ignoradas | Contadas exactamente |
| **Términos menores** | Ignorados | Incluidos |
| **Uso principal** | Análisis de algoritmos | Optimización práctica |

### $\color{#dda}{\text{3.2 Relación Matemática}}$

Big O proporciona la **forma funcional** de los FLOPs:

$$
\text{Big O: } f(n) = O(g(n)) \implies \Phi(f) = c \cdot g(n) + \text{términos menores}
$$

**Donde:**
- $\Phi(f)$ = FLOPs exactos
- $c$ = constante que Big O ignora pero FIC captura
- $g(n)$ = función de crecimiento (ej: $n^2$, $n^3$)

**Ejemplo concreto:**

```
Multiplicación de matrices (n×n)
├─ Big O: O(n³) → "escala cúbicamente"
└─ FLOPs: 2n³ → para n=100: 2,000,000 operaciones exactas

FIC usa el valor exacto (2,000,000), no la forma asintótica
```

### $\color{#dda}{\text{3.3 Por Qué Big O No Es Suficiente para Selección de Modelos}}$

**Caso 1: Misma complejidad asintótica, diferentes FLOPs**

| Operación | Big O | FLOPs reales |
|-----------|-------|---------------|
| Suma de arrays | $O(n)$ | $n$ |
| Producto punto | $O(n)$ | $2n$ |

Big O: ambos son $O(n)$ → "igual complejidad"  
FLOPs: producto punto es 2× más costoso → FIC lo detecta

**Caso 2: Constantes multiplicativas grandes**

```
Algoritmo A: 1000n² operaciones  → O(n²)
Algoritmo B: n³ operaciones      → O(n³)

Para n=10:
├─ Big O dice: "B es peor (cúbico > cuadrático)"
├─ Algoritmo A: 100,000 FLOPs
└─ Algoritmo B: 1,000 FLOPs

FIC selecciona B correctamente (100× más eficiente para n=10)
```

**Caso 3: Términos de orden menor relevantes**

$$
\Phi(f) = 2n^3 + 5n^2 + 100n
$$

- Big O: $O(n^3)$ → ignora $5n^2 + 100n$
- Para $n=10$: términos "menores" = 1,500 (43% del total!)
- FIC: usa 3,500 FLOPs exactos

### $\color{#dda}{\text{3.4 FIC Captura lo que Big O Ignora}}$

$$
\boxed{
\text{FIC captura:}
\begin{cases}
\text{Constantes multiplicativas} & (2n^3 \text{ vs } 5n^3) \\
\text{Términos de orden menor} & (n^3 + n^2 \text{ vs } n^3) \\
\text{Costo exacto para } n \text{ dado} & (1M \text{ vs } 5M \text{ FLOPs}) \\
\text{Arquitectura real} & (\text{depth, width, connections})
\end{cases}
}
$$

### $\color{#dda}{\text{3.5 Complementariedad}}$

**Big O y FLOPs son complementarios, no excluyentes:**

| Etapa | Herramienta | Uso |
|-------|-------------|-----|
| **Diseño algorítmico** | Big O | Elegir clase de algoritmo (ej: $O(n\log n)$ > $O(n^2)$) |
| **Implementación** | FLOPs | Comparar implementaciones específicas |
| **Optimización** | FLOPs + FIC | Seleccionar arquitectura óptima |
| **Deployment** | FLOPs + FIC | Decisión final costo-beneficio |

**Workflow recomendado:**

```
1. Big O → Elegir familia de algoritmos
   (preferir O(n log n) sobre O(n²) para n grande)
   
2. FLOPs → Comparar implementaciones concretas
   (entre dos O(n²), elegir la de menos FLOPs)
   
3. FIC → Balancear precisión vs eficiencia
   (¿vale la pena 10× más FLOPs por 1% más accuracy?)
```

### $\color{#dda}{\text{3.6 Ejemplo en Deep Learning}}$

**Comparación ResNet vs MobileNet:**

```
Big O (ambos):
├─ Complejidad: O(H × W × C × D)
└─ Conclusión: "Escalan igual"

FLOPs (diferentes):
├─ ResNet-50:    4.1 GFLOPs
└─ MobileNetV2:  0.3 GFLOPs
└─ Ratio: 13.7× diferencia

FIC (considera ambos):
├─ ResNet: mejor accuracy (+2%)
├─ MobileNet: mucho más eficiente (13.7×)
└─ Decisión: ¿2% mejora vale 13.7× más cómputo?
    → FIC cuantifica este trade-off
```

## $\color{#dda}{\text{4. Fórmula Principal (Versión 2.0 o algo así - Paramétrica)}}$


$$
\boxed{
\text{FIC} = -2\log \mathcal{L}(\hat{\theta}|\mathcal{D}) + \alpha \left[\lambda \log(\Phi(f)) + (1-\lambda)\frac{\Phi(f)}{10^6}\right] + \beta \cdot k
}
$$

### Variables Principales:

| Variable | Nombre | Descripción | Unidades | Rango típico |
|----------|--------|-------------|----------|---------------|
| $\text{FIC}$ | FIC score | Valor del criterio (menor = mejor) | real | $[0, \infty)$ |
| $\mathcal{L}(\hat{\theta}\|\mathcal{D})$ | Verosimilitud | Probabilidad de datos dado modelo | $[0, 1]$ | - |
| $\hat{\theta}$ | Parámetros | Estimaciones MLE/MAP | vector $\mathbb{R}^k$ | - |
| $\mathcal{D}$ | Datos | Dataset $\{(x_i, y_i)\}_{i=1}^n$ | conjunto | - |
| $\Phi(f)$ | FLOPs | Operaciones de punto flotante | entero | $10^3$ - $10^{12}$ |
| $f$ | Modelo | Función $f: \mathcal{X} \to \mathcal{Y}$ | función | - |
| $\alpha$ | Peso FLOPs | Penalización computacional | real $\geq 0$ | 2.0 - 10.0 |
| $\lambda$ | Balance log/linear | Mezcla términos FLOPs | $[0, 1]$ | 0.2 - 0.5 |
| $\beta$ | Peso parámetros | Penalización paramétrica | real $\geq 0$ | 0.3 - 1.0 |
| $k$ | Parámetros | Número de parámetros entrenables | entero | $10^3$ - $10^9$ |
| $n$ | Tamaño muestra | Observaciones en $\mathcal{D}$ | entero | $10^2$ - $10^7$ |

### Valores Recomendados (basados en experimentos):

| Escenario | $\alpha$ | $\beta$ | $\lambda$ | Uso |
|-----------|----------|---------|-----------|-----|
| **Móvil/Edge** | 7.0-10.0 | 0.3 | 0.2 | Penaliza fuerte FLOPs |
| **Balanceado** | 5.0 | 0.5 | 0.3 | Default recomendado |
| **Servidor** | 2.0 | 1.0 | 0.5 | Menos restrictivo |

## $\color{#dda}{\text{5. Componentes del FIC Explicados}}$

### $\color{#dda}{\text{5.1 Bondad de Ajuste (Término de Verosimilitud)}}$

$$
\mathcal{L}_{\text{term}} = -2\log \mathcal{L}(\hat{\theta}|\mathcal{D})
$$

**¿Qué mide?** Qué tan bien el modelo explica los datos observados.

**Interpretación:**
- **Menor valor** → mejor ajuste a los datos
- Es idéntico al término de ajuste en AIC/BIC
- No penaliza complejidad, solo mide precisión

**Variables específicas:**

| Variable | Descripción |
|----------|-------------|
| $\log \mathcal{L}(\hat{\theta}\|\mathcal{D})$ | Log-verosimilitud del modelo en los datos |
| Factor -2 | Convención estadística (escala de deviance) |

#### Para Regresión:
$$
-2\log \mathcal{L} = n\log\left(\frac{\text{RSS}}{n}\right) + n\log(2\pi) + n
$$

| Variable | Descripción |
|----------|-------------|
| $\text{RSS}$ | $\sum_{i=1}^n (y_i - \hat{y}_i)^2$ (suma residuos cuadrados) |
| $y_i$ | Valor observado |
| $\hat{y}_i$ | Predicción del modelo |

#### Para Clasificación:
$$
-2\log \mathcal{L} = -2\sum_{i=1}^{n} \log P(y_i|x_i, \hat{\theta})
$$

| Variable | Descripción |
|----------|-------------|
| $P(y_i\|x_i, \hat{\theta})$ | Probabilidad asignada a clase correcta |
| Rango | $[0, 1]$ por observación |

**Por qué importa:** Un modelo que predice mal tendrá -2log(L) alto, penalizándolo automáticamente incluso si es eficiente.

### $\color{#dda}{\text{5.2 Término de Penalización Computacional (REMI PERO QUÉ NOVEDOSO)}}$

$$
\mathcal{C}_{\text{comp}} = \alpha \left[\lambda \log(\Phi(f)) + (1-\lambda)\frac{\Phi(f)}{10^6}\right]
$$

**¿Qué mide?** Costo computacional real de ejecutar el modelo.

**Por qué es híbrido (log + lineal)?**

| Componente | Fórmula | Sensibilidad | Cuándo domina |
|------------|---------|--------------|----------------|
| **Logarítmico** | $\lambda \log(\Phi)$ | Diferencias relativas | FLOPs pequeños/medios |
| **Lineal** | $(1-\lambda)\frac{\Phi}{10^6}$ | Diferencias absolutas | FLOPs grandes |

**Ejemplo numérico:**

```python
α = 5.0, λ = 0.3

Modelo A: 1M FLOPs
├─ Término log:    0.3 × log(1,000,000) ≈ 4.14
├─ Término lineal: 0.7 × 1.0 = 0.7
└─ Penalización:   5.0 × (4.14 + 0.7) = 24.2

Modelo B: 10M FLOPs (10× más)
├─ Término log:    0.3 × log(10,000,000) ≈ 4.84  (+0.7)
├─ Término lineal: 0.7 × 10.0 = 7.0  (+6.3)
└─ Penalización:   5.0 × (4.84 + 7.0) = 59.2  (+35.0)

Diferencia: 35 puntos de penalización
→ El término lineal detecta la diferencia de 9M FLOPs
→ El término log suaviza para no sobre-penalizar
```

**Parámetro $\lambda$ (balance):**

| $\lambda$ | Comportamiento | Uso |
|-----------|----------------|-----|
| 1.0 | 100% logarítmico | Diferencias relativas (ej: 10×) |
| 0.5 | Balance 50/50 | Propósito general |
| 0.3 | 70% lineal | **Recomendado**: detecta diferencias absolutas |
| 0.0 | 100% lineal | Penalización muy agresiva |

**Variables específicas:**

| Variable | Descripción | Cálculo |
|----------|-------------|---------|
| $\Phi(f)$ | FLOPs totales | Automático vía `flop_counter` |
| $\alpha$ | Peso global | Controla cuánto importan FLOPs vs ajuste |
| $\lambda$ | Balance | Mezcla sensibilidad relativa/absoluta |
| $10^6$ | Normalización | Escala a MegaFLOPs para el término lineal |

**Cálculo de $\Phi(f)$ según operación:**

#### Multiplicación de Matrices ($A \in \mathbb{R}^{m \times k}$, $B \in \mathbb{R}^{k \times n}$):
$$
\Phi_{\text{matmul}} = 2mnk
$$

#### Convolución 2D:
$$
\Phi_{\text{conv2d}} = 2 \cdot H' \cdot W' \cdot C_{\text{out}} \cdot C_{\text{in}} \cdot K_h \cdot K_w
$$

| Variable | Descripción |
|----------|-------------|
| $H', W'$ | Dimensiones de salida |
| $C_{\text{in}}, C_{\text{out}}$ | Canales entrada/salida |
| $K_h, K_w$ | Tamaño del kernel |

#### Red Feed-Forward (L capas):
$$
\Phi_{\text{NN}} = 2\sum_{l=1}^{L} d_{l-1} \cdot d_l
$$

| Variable | Descripción |
|----------|-------------|
| $d_l$ | Neuronas en capa $l$ |

**Por qué importa:** 
- AIC/BIC ignoran completamente este término
- Dos modelos con igual # de parámetros pero diferentes FLOPs son equivalentes para AIC/BIC
- FIC los distingue correctamente

### $\color{#dda}{\text{5.3 Término de Penalización Paramétrica}}$

$$
\mathcal{C}_{\text{param}} = \beta \cdot k
$$

**¿Qué mide?** Complejidad del modelo en términos de capacidad de expresión.

**Interpretación:**
- Previene overfitting (muchos parámetros → sobreajuste)
- Correlaciona con uso de memoria (más params → más RAM)
- Idéntico al término de penalización en AIC/BIC (y de alguna manera mágica no es plagio :3)

**Variables:**

| Variable | Descripción | Ejemplo |
|----------|-------------|----------|
| $k$ | Parámetros entrenables | Pesos + biases |
| $\beta$ | Peso de penalización | 0.5 (default), 1.0 (AIC-like) |

**Cálculo de $k$ por tipo de modelo:**

#### Regresión Lineal:
$$
k = p + 1
$$
(p predictores + 1 intercept)

#### Red Neuronal:
$$
k = \sum_{l=1}^{L} (d_{l-1} \cdot d_l + d_l)
$$
(pesos de conexiones + biases)

#### CNN:
$$
k = \sum_{\text{capas}} (C_{\text{in}} \cdot C_{\text{out}} \cdot K_h \cdot K_w + C_{\text{out}})
$$

**Por qué $\beta$ es menor que 1.0 en FIC?**

```
Configuración tradicional (AIC): β = 2, α = 0
→ Solo penaliza parámetros

Configuración FIC (recomendada): β = 0.5, α = 5.0
→ Da más peso a FLOPs que a parámetros
→ Razón: en deployment, cómputo importa más que memoria
```

**SIN EMBARGO**
```
Se puede utilizar cualquier proporción, sin importar si β > 1
→ Penalizando mucho más los parámetros

En realidad, de esta manera también puede contemplarse memoria
→ Si nuestro objetivo es no solo reducir la cantidad de costo computacional, sino también memoria
→ Pueden darse casos en los que memoria importe más que coste computacional
```

Los parámetros de la fórmula que diseñé se basan completamente en lo que se busque ahorrar al elegir un modelo así que, a pesar de que este notebook busque escatimar en poder de cómputo, también se puede usar de forma más "clásica" buscando ahorrar en parámetros (y por tanto, en memoria)

## $\color{#dda}{\text{6. Configuraciones del FIC}}$


Basándome en los resultados del notebook de refinado (fic_refining.ipynb) y los experimentos que hice, llegué a la conclusión de que los parámetros dependerán un poco de los modelos a optimizar o comparar. De todas formas dejo algunos combos que me gustaron porque demostraron poder usarse de forma más general y balanceada.

### $\color{#dda}{\text{6.1 FIC-Balanceado (Recomendado)}}$

$$
\text{FIC}_{\text{bal}} = -2\log \mathcal{L} + 5.0 \left[0.3\log(\Phi)+ 0.7\frac{\Phi}{10^6}\right] + 0.5k
$$

**Configuración:**

| Parámetro | Valor | Razón |
|-----------|-------|-------|
| $\alpha$ | 5.0 | Penalización moderada-alta de FLOPs |
| $\beta$ | 0.5 | Menos peso a parámetros (FLOPs más críticos) |
| $\lambda$ | 0.3 | 30% log, 70% lineal (detecta diferencias grandes) |

**Uso:** Default para producción. Balance entre precisión y eficiencia.

**Expandida:**
$$
\text{FIC}_{\text{bal}} = -2\log \mathcal{L} + 1.5\log(\Phi) + 3.5\frac{\Phi}{10^6} + 0.5k
$$

### $\color{#dda}{\text{6.2 FIC-Móvil (Muy Restrictivo)}}$

$$
\text{FIC}_{\text{móvil}} = -2\log \mathcal{L} + 10.0 \left[0.2\log(\Phi) + 0.8\frac{\Phi}{10^6}\right] + 0.3k
$$

**Configuración:**

| Parámetro | Valor | Razón |
|-----------|-------|-------|
| $\alpha$ | 10.0 | Penalización muy agresiva de FLOPs |
| $\beta$ | 0.3 | Mínimo peso a parámetros |
| $\lambda$ | 0.2 | 80% lineal (muy sensible a FLOPs absolutos) |

**Uso:** Móviles, edge devices, IoT. Batería y latencia son críticas.

**Expandida:**
$$
\text{FIC}_{\text{móvil}} = -2\log \mathcal{L} + 2.0\log(\Phi) + 8.0\frac{\Phi}{10^6} + 0.3k
$$

**Efecto:** Modelo con 10M FLOPs vs 1M FLOPs tiene penalización adicional de ~72 puntos.

### $\color{#dda}{\text{6.3 FIC-Servidor (Menos Restrictivo)}}$

$$
\text{FIC}_{\text{servidor}} = -2\log \mathcal{L} + 2.0 \left[0.5\log(\Phi) + 0.5\frac{\Phi}{10^6}\right] + 1.0k
$$

**Configuración:**

| Parámetro | Valor | Razón |
|-----------|-------|-------|
| $\alpha$ | 2.0 | Penalización suave (análoga a AIC) |
| $\beta$ | 1.0 | Peso completo a parámetros |
| $\lambda$ | 0.5 | Balance 50/50 |

**Uso:** Servidores con recursos abundantes. Precisión es prioridad.

**Expandida:**
$$
\text{FIC}_{\text{servidor}} = -2\log \mathcal{L} + 1.0\log(\Phi) + 1.0\frac{\Phi}{10^6} + 1.0k
$$

### $\color{#dda}{\text{6.4 Comparación Visual}}$

```
Modelo A: 1M FLOPs, 100K params, -2log(L) = 1000
Modelo B: 10M FLOPs, 100K params, -2log(L) = 990

Análisis:
├─ Modelo B es 1% mejor en ajuste (990 vs 1000)
└─ Modelo B es 10× más costoso (10M vs 1M FLOPs)

Configuración Móvil (α=10, λ=0.2):
├─ FIC(A) = 1000 + 2.0×log(1M) + 8.0×1.0 + 30 = 1065.8
├─ FIC(B) = 990 + 2.0×log(10M) + 8.0×10.0 + 30 = 1135.8
└─ Δ = +70 → Selecciona A (B no vale la pena)

Configuración Servidor (α=2, λ=0.5):
├─ FIC(A) = 1000 + 1.0×log(1M) + 1.0×1.0 + 100 = 1114.8
├─ FIC(B) = 990 + 1.0×log(10M) + 1.0×10.0 + 100 = 1116.3
└─ Δ = +1.5 → Ambos comparables
```

## $\color{#dda}{\text{7. Selección de Modelo}}$


### $\color{#dda}{\text{7.1 Criterio de Decisión}}$

Dado un conjunto de modelos candidatos $\mathcal{F} = \{f_1, f_2, \ldots, f_M\}$:

$$
f^* = \arg\min_{f \in \mathcal{F}} \text{FIC}(f)
$$

**Variables:**

| Variable | Descripción |
|----------|-------------|
| $\mathcal{F}$ | Conjunto de modelos candidatos |
| $M$ | Número de modelos |
| $f^*$ | Modelo óptimo (menor FIC) |

**Principio:** Menor FIC = mejor balance entre ajuste, eficiencia y simplicidad.

### $\color{#dda}{\text{7.2 Diferencia de FIC (ΔFIC)}}$

Para comparar dos modelos $f_1$ y $f_2$:

$$
\Delta\text{FIC} = \text{FIC}(f_1) - \text{FIC}(f_2)
$$

**Interpretación (Guidelines de Burnham & Anderson):**

| $|\Delta\text{FIC}|$ | Evidencia | Decisión |
|-------------------|-----------|----------|
| 0 - 2 | Sustancialmente equivalentes | Ambos modelos comparables |
| 2 - 10 | Evidencia considerable | Preferir modelo con menor FIC |
| > 10 | Evidencia muy fuerte | Modelo con mayor FIC es claramente inferior |

**Ejemplo práctico:**

```
FIC(MLP) = 1000
FIC(CNN) = 1005

ΔFIC = 5 → Evidencia considerable contra CNN
Interpretación: CNN no justifica su complejidad adicional
Decisión: Usar MLP
```

## $\color{#dda}{\text{8. Estimación de Hiperparámetros α, β, λ}}$

### $\color{#dda}{\text{8.1 Mediante Validación Cruzada}}$

$$
(\alpha^*, \beta^*, \lambda^*) = \arg\min_{(\alpha, \beta, \lambda) \in \Lambda} \text{CV-Error}(\alpha, \beta, \lambda)
$$

**Espacio de búsqueda:**
$$
\Lambda = \{(\alpha, \beta, \lambda) : \alpha \in [0.5, 10], \beta \in [0, 2], \lambda \in [0, 1]\}
$$

**Procedimiento:**

1. Definir grids:
   - $\alpha \in \{0.5, 1.0, 2.0, 5.0, 7.0, 10.0\}$
   - $\beta \in \{0.0, 0.3, 0.5, 1.0, 2.0\}$
   - $\lambda \in \{0.0, 0.2, 0.3, 0.4, 0.5, 0.8, 1.0\}$

2. Para cada combinación:
   - Calcular FIC de todos los modelos
   - Seleccionar mejor modelo según FIC
   - Evaluar en validación

3. Elegir $(\alpha^*, \beta^*, \lambda^*)$ que minimizan error de validación

**Implementación:** Ver `fic_parameter_optimization.ipynb`

### $\color{#dda}{\text{8.2 Valores Recomendados (Empíricos)}}$

Basados en experimentos con 3 tipos de problemas:

| Escenario | $\alpha$ | $\beta$ | $\lambda$ | Sensibilidad |
|-----------|----------|---------|-----------|---------------|
| **Regresión** | 5.0 | 0.5 | 0.3 | Alta (ΔFIC ~25) |
| **Redes Neuronales** | 5.0 | 0.5 | 0.3 | Media (ΔFIC ~150) |
| **Polinomios** | 5.0 | 0.5 | 0.3 | Muy alta (ΔFIC ~700) |

**Conclusión:** $(\alpha=5.0, \beta=0.5, \lambda=0.3)$ es robusto across diferentes problemas.

**Alternativas por analogía:**

#### Análoga a AIC:
$$
\alpha = 2.0, \quad \beta = 0.0, \quad \lambda = 0.5
$$

#### Análoga a BIC:
$$
\alpha = \log(n), \quad \beta = 0.0, \quad \lambda = 0.5
$$

## $\color{#dda}{\text{9. Propiedades Matemáticas del FIC}}$


### $\color{#dda}{\text{9.1 Consistencia Asintótica}}$

**Definición:** El FIC es consistente si selecciona el modelo verdadero cuando $n \to \infty$:

$$
\lim_{n \to \infty} P(\text{FIC selecciona } f_0) = 1
$$

**Condiciones suficientes:**
- $\alpha = o(\sqrt{n})$ → penalización crece más lento que $\sqrt{n}$
- $\beta = o(\sqrt{n})$
- $f_0 \in \mathcal{F}$ → modelo verdadero está en candidatos

**Interpretación:** Con suficientes datos, FIC encuentra el modelo correcto.

### $\color{#dda}{\text{9.2 Invarianza a Escala de FLOPs}}$

Si transformamos FLOPs por constante $c > 0$:

$$
\Phi'(f) = c \cdot \Phi(f)
$$

Entonces:

$$
\text{FIC}'(f) = \text{FIC}(f) + \alpha\lambda\log(c) + \alpha(1-\lambda)\frac{c-1}{10^6}\Phi(f)
$$

**Conclusión:** El **ranking relativo** se preserva (todos los modelos se desplazan igual).

**Implicación práctica:** No importa si contamos FLOPs con o sin ciertos overheads, el orden de modelos no cambia.

## $\color{#dda}{\text{10. Casos de Uso Específicos}}$


### $\color{#dda}{\text{10.1 Regresión Lineal}}$

**Modelo:**
$$
y_i = \beta_0 + \beta_1 x_i + \epsilon_i, \quad \epsilon_i \sim \mathcal{N}(0, \sigma^2)
$$

**FIC:**
$$
\text{FIC} = n\log(\text{RSS}) + \alpha\left[\lambda\log(3np) + (1-\lambda)\frac{3np}{10^6}\right] + \beta \cdot 2
$$

**Variables:**

| Variable | Valor | Explicación |
|----------|-------|-------------|
| $3np$ | FLOPs | $2np$ (matmul) + $np$ (residuos) |
| $k$ | 2 | $\beta_0, \beta_1$ |
| RSS | $\sum(y_i - \hat{y}_i)^2$ | Suma residuos cuadrados |

### $\color{#dda}{\text{10.2 Red Neuronal Feed-Forward}}$

**Arquitectura:** $[d_0, d_1, d_2, \ldots, d_L]$

**FIC:**
$$
\text{FIC} = -2\sum_{i=1}^n \log P(y_i|x_i) + \alpha\left[\lambda\log\left(2\sum_{l=1}^L d_{l-1}d_l\right) + (1-\lambda)\frac{2\sum_{l=1}^L d_{l-1}d_l}{10^6}\right] + \beta\sum_{l=1}^L (d_{l-1}d_l + d_l)
$$

**Componentes:**

| Término | Fórmula | Interpretación |
|---------|---------|----------------|
| FLOPs | $2\sum d_{l-1}d_l$ | Todas las multiplicaciones matriz-vector |
| Parámetros | $\sum (d_{l-1}d_l + d_l)$ | Pesos + biases |

### $\color{#dda}{\text{10.3 Red Convolucional (CNN)}}$

**Para una capa convolucional:**

$$
\Phi_{\text{conv}} = 2H'W'C_{\text{out}}C_{\text{in}}K_hK_w
$$

$$
k_{\text{conv}} = C_{\text{out}} \cdot C_{\text{in}} \cdot K_h \cdot K_w + C_{\text{out}}
$$

**FIC de la capa:**
$$
\text{FIC}_{\text{capa}} = \alpha\left[\lambda\log(\Phi_{\text{conv}}) + (1-\lambda)\frac{\Phi_{\text{conv}}}{10^6}\right] + \beta \cdot k_{\text{conv}}
$$

**Para arquitectura completa:**
$$
\text{FIC}_{\text{CNN}} = -2\log \mathcal{L} + \alpha\left[\lambda\log\left(\sum_{\text{capas}} \Phi\right) + (1-\lambda)\frac{\sum_{\text{capas}} \Phi}{10^6}\right] + \beta\sum_{\text{capas}} k
$$

## $\color{#dda}{\text{11. Resumen de Notación Completa}}$

| Símbolo | Nombre | Descripción | Tipo | Rango |
|---------|--------|-------------|------|-------|
| $\text{FIC}$ | FIC score | Valor del criterio | escalar | $[0, \infty)$ |
| $\mathcal{L}$ | Verosimilitud | P(datos\|modelo) | $[0,1]$ | - |
| $\hat{\theta}$ | Parámetros | Estimaciones MLE | vector | $\mathbb{R}^k$ |
| $\mathcal{D}$ | Datos | Training set | conjunto | - |
| $\Phi(f)$ | FLOPs | Operaciones FP | entero | $10^3$-$10^{12}$ |
| $f$ | Modelo | Función predictiva | función | - |
| $\alpha$ | Peso FLOPs | Penalización comp. | real $\geq 0$ | 2-10 |
| $\lambda$ | Balance | Mix log/linear | $[0,1]$ | 0.2-0.5 |
| $\beta$ | Peso params | Penalización param. | real $\geq 0$ | 0.3-1.0 |
| $k$ | Parámetros | Dim($\theta$) | entero | $10^3$-$10^9$ |
| $n$ | Muestra | # observaciones | entero | $10^2$-$10^7$ |

## $\color{#dda}{\text{12. Fórmula Final (y mis recomendaciones)}}$

$$\color{#ddd}{\text{Se le pueden poner casi cualquier }{\alpha, \beta  \text{ o }  \lambda}}$$

$$
\boxed{
\text{FIC} = -2\log \mathcal{L}(\hat{\theta}|\mathcal{D}) + \alpha \left[\lambda \log(\Phi(f)) + (1-\lambda)\frac{\Phi(f)}{10^6}\right] + \beta \cdot k
}
$$

$$\color{#ddd}{\text{Pero aquí está mi receta para comenzar un análisis de viabilidad computacional}}$$

$$
\boxed{
\begin{aligned}
\text{FIC} &= -2\log \mathcal{L}(\hat{\theta}|\mathcal{D}) + 5.0\left[0.3\log(\Phi(f)) + 0.7\frac{\Phi(f)}{10^6}\right] + 0.5k \\
&= \underbrace{-2\log \mathcal{L}}_{\text{ajuste a datos}} + \underbrace{1.5\log(\Phi) + 3.5\frac{\Phi}{10^6}}_{\text{penalización computacional}} + \underbrace{0.5k}_{\text{penalización paramétrica}}
\end{aligned}
}
$$

**Componentes:**
1. **Primer término ($-2\log \mathcal{L}$):** Mide ajuste (menor = mejor fit)
2. **Segundo término ($1.5\log(\Phi) + 3.5\frac{\Phi}{10^6}$):** Penaliza FLOPs altos
   - Parte log: sensible a diferencias relativas
   - Parte lineal: sensible a diferencias absolutas
3. **Tercer término ($0.5k$):** Penaliza muchos parámetros (overfitting + memoria)

**Interpretación unificada:**
> Selecciona el modelo que mejor balancea precisión, eficiencia computacional y simplicidad.

**Si no vas a usar mi fórmula mínimo ten en cuenta los FLOPs, la electricidad de por sí ya está bien cara**
```
AIC = -2log(L) + 2k            → Ignora FLOPs completamente
BIC = -2log(L) + k·log(n)      → Ignora FLOPs completamente
FIC = -2log(L) + f(FLOPs) + βk → Considera FLOPs + parámetros
```